In [1]:
flower_mapping = {'daisy':0, 'dandelion':1, 'rose':2, 'sunflower':3, 'tulip':4}
train_dir = "ml100-03-final/image_data/train/"
test_dir = "ml100-03-final/image_data/test/test"

# 訓練用的超參數
batch_size = 256
epochs = 8
# data_augmentation = True
num_classes = 5

# 資料標準化的方式，此處使用減去所有影像的平均值
# subtract_pixel_mean = True

img_shape = (224, 224)

In [2]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    validation_split=0.1,
    rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
)

Using TensorFlow backend.


In [3]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_shape,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_shape,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
)

Found 2543 images belonging to 5 classes.
Found 280 images belonging to 5 classes.


In [4]:
import numpy as np
num_classes = len(np.unique(train_generator.classes))
input_shape = train_generator.target_size+(3,)
print(num_classes, img_shape, input_shape)

5 (224, 224) (224, 224, 3)


In [5]:
from keras.models import Model
from keras.layers import Input

input_tensor = Input(shape=input_shape)

# from keras.applications.vgg16 import VGG16
# model = VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)

# from keras.applications.nasnet import NASNetMobile
# model = NASNetMobile(
#     input_shape=None,
#     include_top=True,
# #     weights='imagenet',
#     weights=None,
#     input_tensor=input_tensor,
#     pooling=None,
#     classes=num_classes
# )

from keras.applications.inception_v3 import InceptionV3
model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

model.layers.pop()
model.outputs = [model.layers[-1].output]
model.layers[-1].outbound_nodes = []

from keras.layers import Dense, Dropout, Flatten
x = model.output
x = Flatten(name='flatten')(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(num_classes, activation='softmax', name='predictions')(x)

model=Model(inputs=model.input, outputs=x)
print(model.summary())

# for layer in model.layers:
#     layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

87916544/87910968 [==============================] - 350s 4us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
_________________________

conv2d_21 (Conv2D)              (None, 25, 25, 48)   13824       mixed1[0][0]                     
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 25, 25, 96)   55296       activation_23[0][0]              
__________________________________________________________________________________________________
batch_normalization_21 (BatchNo (None, 25, 25, 48)   144         conv2d_21[0][0]                  
__________________________________________________________________________________________________
batch_normalization_24 (BatchNo (None, 25, 25, 96)   288         conv2d_24[0][0]                  
__________________________________________________________________________________________________
activation_21 (Activation)      (None, 25, 25, 48)   0           batch_normalization_21[0][0]     
__________________________________________________________________________________________________
activation

conv2d_53 (Conv2D)              (None, 12, 12, 160)  179200      activation_52[0][0]              
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 12, 12, 160)  179200      activation_57[0][0]              
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 12, 12, 160)  480         conv2d_53[0][0]                  
__________________________________________________________________________________________________
batch_normalization_58 (BatchNo (None, 12, 12, 160)  480         conv2d_58[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 12, 12, 160)  0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
activation

In [6]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//32,
    epochs=epochs,
    shuffle=True,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples//32,
)

Epoch 1/8
79/79 [==============================] - 109731s 1389s/step - loss: 5.1481 - accuracy: 0.2419 - val_loss: 1.6430 - val_accuracy: 0.2429
Epoch 2/8
79/79 [==============================] - 43458s 550s/step - loss: 0.8591 - accuracy: 0.6289 - val_loss: 1.5008 - val_accuracy: 0.7357
Epoch 3/8
79/79 [==============================] - 40117s 508s/step - loss: 0.1252 - accuracy: 0.9645 - val_loss: 4.0346 - val_accuracy: 0.4821
Epoch 4/8
79/79 [==============================] - 41290s 523s/step - loss: 0.0921 - accuracy: 0.9778 - val_loss: 0.2234 - val_accuracy: 0.8929
Epoch 5/8
79/79 [==============================] - 41669s 527s/step - loss: 0.0337 - accuracy: 0.9927 - val_loss: 1.4246 - val_accuracy: 0.8321
Epoch 6/8
79/79 [==============================] - 40818s 517s/step - loss: 0.0648 - accuracy: 0.9871 - val_loss: 0.2714 - val_accuracy: 0.8607
Epoch 7/8
79/79 [==============================] - 44378s 562s/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.8879 - val_accurac

In [ ]:
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

In [7]:
test_generator = ImageDataGenerator(
    rescale=1./255,
).flow_from_directory(
    test_dir,
    target_size=img_shape,
    batch_size=1,
    class_mode=None,
    shuffle=False,
)

Found 0 images belonging to 0 classes.


In [8]:
test_generator.reset()
pred = model.predict_generator(test_generator, verbose=1)
predicted_class_indices = np.argmax(pred, axis=1)
labels = (train_generator.class_indices)
label = dict((v,k) for k,v in labels.items())
predictions = [label[i] for i in predicted_class_indices]
filenames = [s[5:-4] for s in test_generator.filenames]

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
train_generator.class_indices

In [ ]:
predictions

In [ ]:
# model.save("4.l54a84.h5")
# from keras.models import load_model
# from keras.models import Sequential
# model = Sequential()
# model = load_model("1.h5")

In [ ]:
import pandas as pd
from datetime import datetime
now = datetime.today().strftime('%Y%m%d%H%M%S')
mname = type(model).__name__
print(mname, now)
pd.DataFrame({'id': filenames, 'flower_class': predicted_class_indices}).to_csv('JL_'+mname+'.'+now+'model6.csv', index=False)